This notebook contains hacks which allows to quantize and prepare models for mobile and test how fast they are working,
but for the real use some extra work is needed.

NeMo quantization cannot be used, because it exports models to TensorRT which does not work on android.

In [ ]:
from argparse import ArgumentParser
from tqdm import tqdm

import torch

from nemo.collections.asr.models import EncDecCTCModel
from nemo.collections.asr.models import EncDecCTCModelBPE
from nemo.utils import logging
from nemo.collections.asr.metrics.wer import WER, word_error_rate

from nemo.collections.asr.parts.submodules.jasper import MaskedConv1d, _masked_conv_init_lens
import torch.nn as nn

import torch.quantization.quantize_fx as quantize_fx

from nemo.core import typecheck
typecheck.set_typecheck_enabled(False)

#import warnings
#warnings.simplefilter("ignore", UserWarning)

from nemo import __version__ as nemo_version

In [ ]:
c = '../nemo_experiments/Citrinet-256-8x-Stride-ru/2022-02-14_00-06-13/checkpoints/Citrinet-256-8x-Stride-ru.nemo'
data = '/media/storage/yashkin/golos/train/1hour.jsonl'

test_ds_l = ['/media/storage/yashkin/golos/test/crowd/test_crowd.jsonl', '/media/storage/yashkin/golos/test/farfield/test_farfield.jsonl']

In [ ]:
nemo_version

In [ ]:
class Minimal(nn.Module):
    def __init__(self, preprocessor, encoder, decoder):
        super().__init__()
        self.preprocessor = preprocessor
        self.encoder = encoder
        self.decoder = decoder

    def forward(self):
        input_signal, input_signal_length = torch.zeros(1, 16000), torch.Tensor([16000])
        pre = self.preprocessor(input_signal, input_signal_length)
        enc = self.encoder(*pre)
        dec = self.decoder(enc[0])
        return dec


can_gpu = False # torch.cuda.is_available()

In [ ]:
def get_forward(mask):
    def forward_t(x, lens):
        self = mask
        #if self.use_mask:
        #    x, lens = self.update_masked_length(x, lens)

        # asymmtric pad if necessary
        if self.pad_layer is not None:
            x = self.pad_layer(x)

        sh = x.shape
        if self.heads != -1:
                x = x.view(-1, self.heads, sh[-1])

        out = self.conv(x)

        if self.heads != -1:
            out = out.view(sh[0], self.real_out_channels, -1)

        return out, lens
    return forward_t
    


def make_traceable(model):
    assert nemo_version == '1.5.0', 'forward_t will not work with other NeMo versions'
    
    for name, module in model.named_children():
        if len(list(module.children())) > 0:
            ## compound module, go inside it
            make_traceable(module)

        if isinstance(module, MaskedConv1d):
            module.use_mask = False
            module.forward = get_forward(module)

In [ ]:
# asr_model = EncDecCTCModelBPE.restore_from(c)
asr_model = EncDecCTCModel.from_pretrained("QuartzNet15x5Base-En")

In [ ]:
def setup_data(asr_model, vocabulary, data):
    asr_model.preprocessor.featurizer.pad_to = 0
    asr_model.preprocessor.featurizer.dither = 0.0
    
    asr_model.setup_test_data(
        test_data_config={
            'sample_rate': 16000,
            'manifest_filepath': data,
            'labels': vocabulary,
            'batch_size': 64,
            'normalize_transcripts': False,
            'num_workers': 4,
        }
    )

In [ ]:
vocabulary = asr_model.decoder.vocabulary
setup_data(asr_model, vocabulary, data)

if can_gpu:
    asr_model = asr_model.cuda()

make_traceable(asr_model)
asr_model.eval();

In [ ]:
minimal = Minimal(asr_model.preprocessor, asr_model.encoder, asr_model.decoder)
minimal.cpu()
# FIXME !!!
trace = torch.jit.trace(minimal, (), check_trace=False)

Save float version

In [ ]:
#trace._save_for_lite_interpreter("trace_q_f.ptl")

In [ ]:
qconfig_dict = qconfig_dict = {"": torch.quantization.get_default_qconfig('qnnpack')}

In [ ]:
asr_model.encoder = quantize_fx.prepare_fx(asr_model.encoder, qconfig_dict)
asr_model.decoder = quantize_fx.prepare_fx(asr_model.decoder, qconfig_dict)

In [ ]:
for test_batch in asr_model.test_dataloader():
    if can_gpu:
        test_batch = [x.cuda() for x in test_batch]
    log_probs, encoded_len, greedy_predictions = asr_model(
        input_signal=test_batch[0], input_signal_length=test_batch[1]
    )
    del test_batch
    break

In [ ]:
asr_model.encoder = quantize_fx.convert_fx(asr_model.encoder)
asr_model.decoder = quantize_fx.convert_fx(asr_model.decoder)

In [ ]:
asr_model.cpu();

In [ ]:
minimal = Minimal(asr_model.preprocessor, asr_model.encoder, asr_model.decoder)
# FIXME !!!
trace = torch.jit.trace(minimal, (), check_trace=False)

In [ ]:
trace._save_for_lite_interpreter("trace_q_i.ptl")

Save int8 version